In [2]:
import ee
import datetime
import folium
!pip install geemap
import geemap

ee.Initialize()

     |████████████████████████████████| 399kB 6.3MB/s 
     |████████████████████████████████| 143kB 10.3MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 1.1MB 10.6MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 5.1MB 22.1MB/s 
     |████████████████████████████████| 1.9MB 49.9MB/s 
     |████████████████████████████████| 3.3MB 41.5MB/s 
     |████████████████████████████████| 225kB 54.4MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 194kB 47.1MB/s 
     |████████████████████████████████| 552kB 35.3MB/s 
     |████████████████████████████████| 112kB 46.2MB/s 
     |████████████████████████████████| 122kB 54.8MB/s 
     |████████████████████████████████| 430kB 43.7MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 


ImportError: ignored

In [ ]:
ee_date = ee.Date('2020-01-01')
py_date = datetime.datetime.utcfromtimestamp(ee_date.getInfo()['value']/1000.0)
print(py_date)

2020-01-01 00:00:00


In [ ]:
medellin = ee.Image("users/shehryarsaroya/Medellin_s2")
medellin_gt = ee.Image("users/shehryarsaroya/Medellin_ground_truth_s2")
print(medellin)
print(medellin.getInfo())
print(medellin_gt)
print(medellin_gt.getInfo())

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "users/shehryarsaroya/Medellin_s2"
      }
    }
  }
})
{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [143, 101], 'crs': 'EPSG:32618', 'crs_transform': [10, 0, 430110, 0, -10, 691490]}, {'id': 'b5', 'data

In [ ]:
Map = geemap.Map(center=[6.2497,-75.58305], zoom=12)
Map

Map(center=[6.2497, -75.58305], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

In [ ]:
polygons = medellin_gt.reduceToVectors(scale = 10, labelProperty = "informal_settlement")
print(polygons.getInfo())

{'type': 'FeatureCollection', 'columns': {'count': 'Long<0, 4294967295>', 'informal_settlement': 'Short<0, 255>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-75.63184693929067, 6.246428344560953], [-75.6209091231272, 6.246441364463491], [-75.62090922971834, 6.246531821498477], [-75.62099962500375, 6.246531714825989], [-75.62099973161197, 6.246622171859106], [-75.62109012691133, 6.246622065169533], [-75.62109023353665, 6.246712522200782], [-75.62118062884994, 6.246712415494126], [-75.62118073549235, 6.246802872523506], [-75.62136152614526, 6.246802659060498], [-75.62136131279998, 6.246621745007657], [-75.62145170809303, 6.246621638255978], [-75.62145160140724, 6.2465311812306465], [-75.62172278723044, 6.246530860887126], [-75.62172289396278, 6.246621317907782], [-75.62199407981834, 6.246620997419846], [-75.62199418659881, 6.2467114544355145], [-75.62217497719232, 6.246711240696139], [-75.6221750840054,

In [ ]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
bands = ['b2', 'b3', 'b4']
# This property of the table stores the land cover labels.
label = 'informal_settlement'

# Overlay the points on the imagery to get training.
training = medellin.select(bands).sampleRegions(**{
  'collection': polygons,
  'properties': [label],
  'scale': 10
})

In [ ]:
svm_classifier = ee.Classifier.libsvm(kernelType = 'RBF')
trained = svm_classifier.train(training, label, bands)

In [ ]:
result = medellin.select(bands).classify(trained)

In [ ]:
# Display the classification result and the input image.
Map = geemap.Map(center=[6.2497,-75.58305], zoom=12)
#Map.addLayer(medellin, {"bands" : ['b4', 'b3', 'b2'], "max" : 0.5, "gamma" : 2})
#Map.addLayer(polygons, {}, 'training polygons')
#Map.addLayer(result, {"min" : 0, "max" : 1.0, "palette" : ['red', 'green']}, "informal_settlement")
Map

Map(center=[6.2497, -75.58305], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

In [ ]:
Map = geemap.Map(center=[6.2497,-75.58305], zoom=12)
#Map.addLayer(medellin, {"bands" : ['b4', 'b3', 'b2'], "max" : 0.5, "gamma" : 2})
Map.addLayer(result, {"min" : 0, "max" : 1.0, "palette" : ['red', 'green']}, "informal_settlement")
Map.addLayer(polygons, {}, 'training polygons')
Map

Map(center=[6.2497, -75.58305], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

In [ ]:
Map.addLayer(polygons, {}, 'training polygons')
Map

Map(bottom=8096996.0, center=[6.251312221347882, -75.62495827674867], controls=(WidgetControl(options=['positi…

In [ ]:
medellin_ground_det={
  "bands": ['b1'],
  "min" : 0,
  "max" : 1
}
Map.addLayer(medellin_gt,medellin_ground_det,'Medellin')
Map

Map(bottom=8096996.0, center=[6.251312221347882, -75.62495827674867], controls=(WidgetControl(options=['positi…

In [ ]:
kianda = ee.Image("users/shehryarsaroya/Kianda_s2")
kianda_gt = ee.Image("users/shehryarsaroya/Kianda_ground_truth_s2")


In [ ]:
result_kianda = kianda.select(bands).classify(trained)

In [ ]:
polygons_kianda = kianda_gt.reduceToVectors(scale = 10, labelProperty = "informal_settlement")

In [ ]:
#print(polygons_kianda.getInfo())

In [ ]:
Map = geemap.Map(center=[-1.3115,36.7659242], zoom=15)
#Map.addLayer(medellin, {"bands" : ['b4', 'b3', 'b2'], "max" : 0.5, "gamma" : 2})
Map.addLayer(result_kianda, {"min" : 0, "max" : 1.0, "palette" : ['red', 'green']}, "informal_settlement")
Map.addLayer(polygons_kianda, {}, 'training polygons')
Map

Map(center=[-1.3115, 36.7659242], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBu…